In [ ]:
import pandas as pd

from googleapiclient.discovery import build
from google.colab import userdata

from tqdm import tqdm

import time
from googleapiclient.errors import HttpError


In [ ]:
data= pd.read_csv('vdoLinks.csv')

In [ ]:
data.head()

,youtubeId,movieId,title
0,K26_sDKnvMU,1,Toy Story (1995)
1,3LPANjHlPxo,2,Jumanji (1995)
2,rEnOoWs3FuA,3,Grumpier Old Men (1995)
3,j9xml1CxgXI,4,Waiting to Exhale (1995)
4,ltwvKLnj1B4,5,Father of the Bride Part II (1995)


In [ ]:
data.shape

(25623, 3)

In [ ]:
API_KEY = userdata.get('GoogleAPI')


In [ ]:
data['youtubeId'].head()

,youtubeId
0,K26_sDKnvMU
1,3LPANjHlPxo
2,rEnOoWs3FuA
3,j9xml1CxgXI
4,ltwvKLnj1B4


In [ ]:
def get_youtube_data(video_id):
    youtube = build('youtube', 'v3', developerKey=API_KEY)

    # Get video details
    video_response = youtube.videos().list(
        part='snippet,statistics,contentDetails',
        id=video_id
    ).execute()

    if not video_response['items']:
        return None  # Return None if no video found

    video_data = video_response['items'][0]

    # Collect required data
    y_data = {
        'video_id': video_id,
        'description': video_data['snippet']['description'],
        'view_count': video_data['statistics'].get('viewCount', 0),
        'like_count': video_data['statistics'].get('likeCount', 0),
        'dislike_count': video_data['statistics'].get('dislikeCount', 0),  # Note: Dislike count is deprecated
        'comment_count': video_data['statistics'].get('commentCount', 0),
        'duration': video_data['contentDetails']['duration'],
        'favorite_count': video_data['statistics'].get('favoriteCount', 0),
        'comments': []
    }

    # Check if comments are disabled
    if int(y_data['comment_count']) > 0:
        comment_response = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            textFormat='plainText',
            maxResults=100
        ).execute()

        for item in comment_response['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            y_data['comments'].append(comment)

    return y_data


In [ ]:
get_youtube_data(data['youtubeId'][0]) #checking if the function works

In [ ]:
results = []
count=0
for video_id in tqdm(data['youtubeId'], desc="Processing Videos", unit="video"):

    video_data = get_youtube_data(video_id)
    if video_data:
        results.append(video_data)
        time.sleep(1)
    else:
      count+=1


print(count)
results_df = pd.DataFrame(results)


In [ ]:
results = []

video_ids = data['youtubeId'].tolist()

for i in tqdm(range(0, len(video_ids), 50), desc="Processing Videos", unit="batch"):
    batch_ids = video_ids[i:i + 50]

    try:
        video_data = get_youtube_data(batch_ids)
        results.extend(video_data)  # Extend the results list with the batch
    except HttpError as e:
        if e.resp.status == 403 and "quota" in str(e):
            print("Quota exceeded. Waiting for some time before retrying...")
            time.sleep(10)  # Wait for an hour before retrying
            continue  # Skip to the next iteration
        else:
            print(f"An error occurred: {e}")

In [ ]:
results_df = pd.DataFrame(results)

In [ ]:
results_df

,0
0,video_id
1,description
2,view_count
3,like_count
4,dislike_count
...,...
112,dislike_count
113,comment_count
114,duration
115,favorite_count
